## MODELO

#### ABORDAGEM

- A abordagem utilizada será por um modelo de **REGRESSÃO** para prever o preço dos automóveis

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

df = pd.read_csv('C:\\Users\\felip\\OneDrive\\projetos_python\\projects_datascience\\lighthouse_case\\main\\datasets\\raw\\cars_train.csv', sep='\t', encoding='utf-16')

- Os tratamentos nas variáveis foram feitos a fim de simplificar a contrução dos modelos base; 
- Foram selecionadas as principais marcas e modelos de veículos, os anos de fabricação do veículo foram transformados em uma nova variável que segrega os anos em faixas de 5 em 5 anos; 
- Os tipos de veículos, câmbio e cores foram simplificados também; 
- O tratamento realizado para adicionar informações sobre cidades ou estado foi criar uma nova variável chamada região, para simplificar a modelagem futura;

Desse modo, através das análises de correlação e simplificações, foi possível utilizar 18 variáveis (features) + 1 variável alvo (target);

#### Modelagem dos dados

- Criação do modelo de regressão

Para os tratamentos iniciais serão testados três (4) modelos diferentes: Um baseado em regressão linear, outro em KNN, além de RandomForest e XGBOOST;
- Os modelos foram escolhidos através do modo como são feitas as previsões e em termos de complexidade. 
- Para avaliar os resultados, escolheu-se utilizar a raiz erro quadrático médio (RMSE) e o coeficiente de determinação do modelo (R2) como uma demonstração do erro médio na previsão do preço dos automóveis e o ajuste e explicação dos dados que o modelo está obtendo no carro do coeficiente de determinação.

In [2]:
# Transformaçoes nas variáveis 

df.num_fotos.fillna(df.num_fotos.mean(), inplace=True)
df.fillna(0, inplace=True)
df.drop(['id', 'veiculo_alienado', 'elegivel_revisao'], axis=1, inplace=True)

# Tratamentos para os NaN e dados nas colunas
lista_colunas_binarias = ['dono_aceita_troca', 'veiculo_único_dono', 'revisoes_concessionaria', 'ipva_pago', 'veiculo_licenciado', 'garantia_de_fábrica', 'revisoes_dentro_agenda', 'veiculo_alienado']
lista_object = ['blindado', 'tipo_vendedor', 'entrega_delivery', 'troca', 'elegivel_revisao']

def substituir_zero_por_nao(df, lista_colunas):
    for coluna in lista_colunas:
        if coluna in df.columns:
            df[coluna] = df[coluna].apply(lambda x: "Não" if x == 0 else "Sim")
    return df

df = substituir_zero_por_nao(df, lista_colunas_binarias)

# Agrupar marcas com menos de 190 aparições em "Outras" na variável "marca"
marcas_menos_frequentes = df['marca'].value_counts()[df['marca'].value_counts() < 190].index
df['marca'] = df['marca'].map(lambda x: 'Outras' if x in marcas_menos_frequentes else x)

# Agrupar marcas com menos de 360 aparições em "Outras" na variável "modelo"
modelos_menos_frequentes = df['modelo'].value_counts()[df['modelo'].value_counts() < 360].index
df['modelo'] = df['modelo'].map(lambda x: 'Outras' if x in modelos_menos_frequentes else x)

limites = [1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2023]
rotulos = ['1985-1989', '1990-1994', '1995-1999', '2000-2004', '2005-2009', '2010-2014', '2015-2019', '2020-2022']
df['faixa_de_anos'] = pd.cut(df['ano_de_fabricacao'], bins=limites, labels=rotulos, right=False)

df['tipo'] = df['tipo'].replace(['Perua/SW', 'Cupê', 'Minivan'], 'Outros')
df['cambio'] = df['cambio'].replace(['Automatizada', 'Automatizada DCT', 'Semi-automática', 'Automática Sequencial'], 'Outras')
df['cor'] = df['cor'].replace(['Dourado', 'Verde', 'Vermelho'], 'Outras')

regioes = {
    'São Paulo (SP)': 'Sudeste',
    'Rio de Janeiro (RJ)': 'Sudeste',
    'Paraná (PR)': 'Sul',
    'Santa Catarina (SC)': 'Sul',
    'Minas Gerais (MG)': 'Sudeste',
    'Rio Grande do Sul (RS)': 'Sul',
    'Goiás (GO)': 'Centro-Oeste',
    'Bahia (BA)': 'Nordeste',
    'Pernambuco (PE)': 'Nordeste',
    'Alagoas (AL)': 'Nordeste',
    'Paraíba (PB)': 'Nordeste',
    'Rio Grande do Norte (RN)': 'Nordeste',
    'Pará (PA)': 'Norte',
    'Ceará (CE)': 'Nordeste',
    'Amazonas (AM)': 'Norte',
    'Mato Grosso do Sul (MS)': 'Centro-Oeste',
    'Mato Grosso (MT)': 'Centro-Oeste',
    'Acre (AC)': 'Norte',
    'Sergipe (SE)': 'Nordeste',
    'Espírito Santo (ES)': 'Sudeste',
    'Tocantins (TO)': 'Norte',
    'Maranhão (MA)': 'Nordeste',
    'Piauí (PI)': 'Nordeste',
    'Rondônia (RO)': 'Norte',
    'Roraima (RR)': 'Norte'
}

df['regiao'] = df['estado_vendedor'].map(regioes)

- A seguir estão os testes iniciais dos modelos com as principais métricas utilizadas para avaliação, por último apresenta-se uma tabela com o resumo dos resultados.

In [3]:
# Selecionar as variáveis do modelo
variaveis_modelo = ['marca', 'modelo', 'faixa_de_anos', 'hodometro', 'cambio', 'tipo', 'blindado', 'cor',
                    'tipo_vendedor', 'troca', 'dono_aceita_troca',
                    'veiculo_único_dono', 'revisoes_concessionaria', 'veiculo_licenciado', 'ipva_pago',
                    'garantia_de_fábrica', 'revisoes_dentro_agenda', 'regiao', 'preco']

# Filtrar o dataframe com as variáveis selecionadas
df_modelo = df[variaveis_modelo].copy()

colunas_objeto = df_modelo.select_dtypes(include=['object', 'category']).columns
df_modelo = pd.get_dummies(df_modelo, columns=colunas_objeto)

# Dividir o dataframe em conjunto de treinamento e teste
X = df_modelo.drop('preco', axis=1)
y = df_modelo['preco']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o modelo de regressão linear
linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_test)
y_pred_train = linear.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE (Treino):", rmse_train)
print("RMSE (Teste):", rmse_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred)
print("R2 (Treino):", r2_train)
print("R2 (Teste):", r2_test)

RMSE (Treino): 53567.665879186476
RMSE (Teste): 51318.956064780454
R2 (Treino): 0.5748629817837975
R2 (Teste): 0.5849064499341402


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

# Padronizar as variáveis numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include='number'))
X_test_scaled = scaler.transform(X_test.select_dtypes(include='number'))

# Criar o modelo KNN
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Realizar previsões no conjunto de teste
y_pred = knn.predict(X_test_scaled)
y_pred_train = knn.predict(X_train_scaled)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE (Treino):", rmse_train)
print("RMSE (Teste):", rmse_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred)
print("R2 (Treino):", r2_train)
print("R2 (Teste):", r2_test)

RMSE (Treino): 42612.56904398452
RMSE (Teste): 49270.168425444754
R2 (Treino): 0.7309711149535221
R2 (Teste): 0.6173881185976869


In [5]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)
y_pred_train = random_forest.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE (Treino):", rmse_train)
print("RMSE (Teste):", rmse_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred)
print("R2 (Treino):", r2_train)
print("R2 (Teste):", r2_test)

RMSE (Treino): 18740.44429827177
RMSE (Teste): 49037.12305451796
R2 (Treino): 0.947966475754673
R2 (Teste): 0.620999027816443


In [6]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)
y_pred_train = xgb.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE (Treino):", rmse_train)
print("RMSE (Teste):", rmse_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred)
print("R2 (Treino):", r2_train)
print("R2 (Teste):", r2_test)


RMSE (Treino): 36364.26493788945
RMSE (Teste): 46187.93650551364
R2 (Treino): 0.8040825577235501
R2 (Teste): 0.6637614696861855


## Valores aproximados das principais métricas dos modelos avaliados

|Modelo|RMSEC (R$)| RMSEV (R$)|R^2 (%)|
|---|---|---|---|
|Regressão Linear|53567.67|51318.96|58.5|
|Regressão por KNN|42757.07|50058.02|60.5|
|Regressão por RandomForest|18722.76|49102.52|62.0|
|Regressão por XGBoost|36364.26|46249.35|66.3|

#### Avaliação dos resultados iniciais

- A partir da análise dos modelos, percebe-se que o que obteve um menor erro médio quadrático, ou seja, teve um percentual de acerto maior na previsão do preço dos veículos foi o XGBOOST, com erro de validação RMSEV de 46 mil reais e R2 de 66%.

#### Validação do modelo com conjunto de teste externo

- Serão aplicadas as mesmas transformações no conjunto de teste para aplicação do modelo
- Por fim, será gerado um arquivo csv com as previsões

In [77]:
# Carregar o conjunto de testes
df_teste = pd.read_csv('C:\\Users\\felip\\OneDrive\\projetos_python\\projects_datascience\\lighthouse_case\\main\\datasets\\raw\\cars_test.csv', sep='\t', encoding='utf-16')

In [78]:
df_teste.head(1)

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,...,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado
0,13518783164498355150900635905895481162,8.0,NISSAN,VERSA,1.6 16V FLEXSTART V-DRIVE MANUAL,2021,2021.0,20258.0,Manual,4,...,False,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,Garantia de fábrica,NaN,NaN


In [79]:
# Aplicar as transformações nas colunas
df_test = df_teste.copy()
df_test['num_fotos'].fillna(df_test['num_fotos'].mean(), inplace=True)
df_test.fillna(0, inplace=True)
df_test.drop(['id', 'veiculo_alienado', 'elegivel_revisao'], axis=1, inplace=True)

df_test = substituir_zero_por_nao(df_test, lista_colunas_binarias)

# Agrupar marcas com menos de 190 aparições em "Outras" na variável "marca"
marcas_menos_frequentes = df_test['marca'].value_counts()[df_test['marca'].value_counts() < 64].index
df_test['marca'] = df_test['marca'].map(lambda x: 'Outras' if x in marcas_menos_frequentes else x)

# Agrupar marcas com menos de 360 aparições em "Outras" na variável "modelo"
modelos_menos_frequentes = df_test['modelo'].value_counts()[df_test['modelo'].value_counts() < 126].index
df_test['modelo'] = df_test['modelo'].map(lambda x: 'Outras' if x in modelos_menos_frequentes else x)

df_test['faixa_de_anos'] = pd.cut(df_test['ano_de_fabricacao'], bins=limites, labels=rotulos, right=False)
df_test['tipo'] = df_test['tipo'].replace(['Perua/SW', 'Cupê', 'Minivan', 'Conversível'], 'Outros')
df_test['cambio'] = df_test['cambio'].replace(['Automatizada', 'Automatizada DCT', 'Semi-automática', 'Automática Sequencial'], 'Outras')
df_test['cor'] = df_test['cor'].replace(['Dourado', 'Verde', 'Vermelho', 'Azul'], 'Outras')
df_test['regiao'] = df_test['estado_vendedor'].map(regioes)

In [80]:
variaveis_modelo_externo = ['marca', 'modelo', 'faixa_de_anos', 'hodometro', 'cambio', 'tipo', 'blindado', 'cor',
                    'tipo_vendedor', 'troca', 'dono_aceita_troca',
                    'veiculo_único_dono', 'revisoes_concessionaria', 'veiculo_licenciado', 'ipva_pago',
                    'garantia_de_fábrica', 'revisoes_dentro_agenda', 'regiao']

df_test = df_test[variaveis_modelo_externo].copy()

In [81]:
df_test.head(1)

,marca,modelo,faixa_de_anos,hodometro,cambio,tipo,blindado,cor,tipo_vendedor,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,veiculo_licenciado,ipva_pago,garantia_de_fábrica,revisoes_dentro_agenda,regiao
0,NISSAN,Outras,2020-2022,20258.0,Manual,Hatchback,N,Branco,PF,False,Sim,Não,Não,Sim,Sim,Sim,Não,Sudeste


In [82]:
# Aplicar pd.get_dummies
colunas_objeto = df_test.select_dtypes(include=['object', 'category']).columns
df_test_encoded = pd.get_dummies(df_test, columns=colunas_objeto)

# Realizar a previsão
y_pred = xgb.predict(df_test_encoded)

# Criar DataFrame com as previsões
predictions = pd.DataFrame({'id': df_teste['id'], 'preco': y_pred})

# Salvar as previsões em um arquivo CSV
predictions.to_csv('predicted.csv', index=False)

#### Conclusões

- Não foi possível obter uma previsão tão boa com os modelos testados;
    - Deve se discutir com o cliente se há necessidade de melhorar mais a performance utilizando modelos mais robustos com otimização de hiperparâmetros;
        - Testes iniciais realizados não mostraram efeito positivo; 
    - Possíveis melhorias incluem uma seleção de variável mais detalhada utilizando-se de validação cruzada; 